This file is **very similar** to Preliminary-analysis-1.

In [226]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [227]:
data = pd.read_csv('consolidated-1.csv')

In [228]:
time_window = '3min' # This is the time window over which we will average call arrivals.
time_window_size = int(time_window[0])

In [229]:
# This is the time in seconds taken to complete a call(included post-processing).
data['Avg-time'] = data['Avg-time'].fillna(0) 

$\lambda$ is the number of calls arriving in an interval equal to the <code>time_window_size</code>.

In [230]:
data['lambda'] = data['ncalls'].rolling(window=time_window_size).mean() # Number of calls per 3 minutes.
# The rolling window size has nothing to do with the time window.

In [231]:
data.set_index('Create-time', inplace=True)

$\mu$ is the average time to answer a call. The unit is '<code>time_window</code>'.

In [232]:
data['mu'] = data['Avg-time']/(60 * time_window_size) # The term lambda/mu must be unit-less.

The traffic intensity $E$ is in Erlangs.

In [233]:
data['E'] = data['lambda']*data['mu']

In [234]:
def prob_of_wait(E, m):
    """
    E:      traffic
    m:      # agents.
    Reference: https://en.wikipedia.org/wiki/Erlang_(unit)
    """ 
    p = 1
               
    if m > E:   
        try:
            numerator = E**m/math.factorial(m) * m/(m - E)
            denominator = 0

            for i in range(m):
                denominator += E**i/math.factorial(i)

            denominator += numerator

            p = numerator/denominator
        except OverflowError:
            print(f'Overflow due to E = {E}, m = {m}')
        
    return p

def calculate_ASA(wait_prob, mu, nagents, E, avg_time):   
    if nagents > E:
        return wait_prob * mu/(nagents - E)
    else:
        return avg_time/(60 * time_window_size)
    
def find_nagents(E, m):
    start = m

    if math.isnan(E):
        return start
    
    if E > m:
        threshold = 0.8
        #start = int(E)
        while (prob_of_wait(E, start) > threshold) and (start <= E):
            start += 1            
        
    return start

def find_nagents_1(E, m, threshold_asa, mu, avg_time):
    start = m
             
    if not math.isnan(E):        
        wait_prob = prob_of_wait(E, m)        
        while calculate_ASA(wait_prob, mu, start, E, avg_time) > threshold_asa:
            start += 1
            
    return start


In [235]:
data['wait-prob'] = data.apply(lambda r: prob_of_wait(r['E'], r['nagents']), axis=1)

Refer to [point 15](https://www.callcentrehelper.com/erlang-c-formula-example-121281.htm) for the formula to compute ASA

In [236]:
data['asa'] = data.apply(lambda r: calculate_ASA(r['wait-prob'], r['mu'], r['nagents'], r['E'], r['Avg-time']), axis=1)

In [237]:
data.head()

,ncalls,Avg-time,Period,nagents,office-hour,Avg-calls,iso_day_of_week,lambda,mu,E,wait-prob,asa
Create-time,,,,,,,,,,,,
2020-11-23 09:03:00,46,691.500000,2020-11-23,52,True,NaN,1,NaN,3.841667,NaN,1.0,3.841667
2020-11-23 09:06:00,46,614.285714,2020-11-23,52,True,11.333333,1,NaN,3.412698,NaN,1.0,3.412698
2020-11-23 09:09:00,46,766.769231,2020-11-23,52,True,12.000000,1,46.0,4.259829,195.952137,1.0,4.259829
2020-11-23 09:12:00,46,608.800000,2020-11-23,52,True,8.333333,1,46.0,3.382222,155.582222,1.0,3.382222
2020-11-23 09:15:00,46,640.333333,2020-11-23,52,True,8.000000,1,46.0,3.557407,163.640741,1.0,3.557407


In [238]:
data.reset_index(level=0, inplace=True)

In [239]:
X = data.apply(lambda r: find_nagents_1(r['E'], r['nagents'], 4, r['mu'], r['Avg-time']), axis = 1)
Y = X.rolling(window=5).median()
Y.reset_index(drop=True, inplace=True)
data['additional_nagents_4m'] = Y - data['nagents']
data['additional_nagents_4m'] = data['additional_nagents_4m'].clip(lower=0, axis=0)

In [240]:
X = data.apply(lambda r: find_nagents_1(r['E'], r['nagents'], 2, r['mu'], r['Avg-time']), axis = 1)
Y = X.rolling(window=5).median()
Y.reset_index(drop=True, inplace=True)
data['additional_nagents_2m'] = Y - data['nagents']
data['additional_nagents_2m'] = data['additional_nagents_2m'].clip(lower=0, axis=0)

In [241]:
data['additional_nagents_4m_pct'] = data['additional_nagents_4m']/data['nagents'] * 100
data['additional_nagents_2m_pct'] = data['additional_nagents_2m']/data['nagents'] * 100

In [242]:
data.to_csv('wait_prob_1.csv', index=False, float_format = '%.3f')

The next step is to build a time-series model for $E$.

In [243]:
data['cum-mu'] = data['mu'].expanding(min_periods=5).mean()
data['cum-E'] = data['E'].expanding(min_periods=5).mean()
data['cum-Avg-time'] = data['Avg-time'].expanding(min_periods=5).mean()
data['cum-wait-prob'] = data.apply(lambda r: prob_of_wait(r['cum-E'], r['nagents']), axis=1)
data['cum-asa'] = data.apply(lambda r: calculate_ASA(r['cum-wait-prob'], r['cum-mu'], \
                                                     r['nagents'], r['cum-E'], r['cum-Avg-time']), axis=1)

In [244]:
X = data.apply(lambda r: find_nagents_1(r['cum-E'], r['nagents'], 2, r['cum-mu'], r['cum-Avg-time']), axis = 1)
Y = X.rolling(window=5).median()
Y.reset_index(drop=True, inplace=True)
data['cum_additional_nagents_2m'] = Y - data['nagents']
data['cum_additional_nagents_2m'] = data['additional_nagents_2m'].clip(lower=0, axis=0)

In [245]:
def convert_to_mmss(time):
    if math.isnan(time):
        return ''
    else:
        minute = int(time)
        frac = time - minute
        s = 0
        if frac > 0:
            s = 60.0 * frac
            
        s = int(s)
        sec = str(s).zfill(2)

        return f'{minute}:{sec}'
        

In [246]:
data['asa'] = data['asa'].apply(lambda r: convert_to_mmss(r))
data['cum-asa'] = data['cum-asa'].apply(lambda r: convert_to_mmss(r))

In [247]:
data.to_csv('wait_prob_2.csv', index=False, float_format = '%.3f')

In [248]:
data['asa'].head()

0    3:50
1    3:24
2    4:15
3    3:22
4    3:33
Name: asa, dtype: object